# **Behavior Analysis using DeepLabCut:**
This notebook will illustrate how to:

* Create a new DeepLabCut project
* Sample videos to obtain frames
* Label the bodyparts in the frames
* Creating a training dataset
* Training a network
* Evaluating the network
* Analyzing videos


Nath, Mathis et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019. Paper: https://www.nature.com/articles/s41596-019-0176-0

**All the necessary toolkits are already present in the LINdoscope environment. Please follow the instructions for each cell and enjoy !**

In [1]:
import deeplabcut
import sys
import numpy as np
import matplotlib.pyplot as plt

## 1. Create a new project :
You can use this function to create a new project with sub-directories and a basic configuration file in the user defined directory. Otherwise the project is created in the current working directory.

In [ ]:
task='Behavior_analysis' # Enter the name of your experiment Task
experimenter='LINdoscope' # Enter the name of the experimenter
working_directory= '\\linstore01\public\CogEm\Charitha-Empathy' # Enter the path to the working directory where you want to create the project
videos=['videos/video1.mp4','videos/video2.mp4'] # Enter the paths of your videos OR FOLDER you want to grab frames from

path_config_file=deeplabcut.create_new_project(task,experimenter,videos,working_folder,copy_videos=True, videotype=".mp4", multianimal=False) 
# The function returns the path, where your project is
# Check the folder to see if the config file has been created

## Edit the config.yaml files
**The parameters that you can adjust are:**

* Number of frames to extract for a video (change the 'numframes2pick' parameter)
* Labels for the bodypart markers (list the bodyparts you would like to label. eg. tailtip, eartipleft etc.)
* Size of the markers (change the 'dotsize' parameter)
* p cutoff value (you want this value to be high, this ensures the network plots the points it is confident about)
* Croping of the video (you can also crop the video by setting the 'crop' parameter to 'True' and then adjusting the x1,x2,y1 and y2 parameters)

## 2. Extract frames from the video :
You can use this function to extract frames from the videos to label the bodyparts. A succesfulfeature detector requires a diverse selection of frames. The frames can be extracted using uniform sampling, k-means sampling or manual selection. This is a good point to also check how the cropped frames look, if you have chosen to crop them.

In [ ]:
%matplotlib inline

extracted_frame = deeplabcut.extract_frames(path_config_file,  mode="automatic", algo="kmeans") # Automatic k-means frame extraction. Set algo='uniform' to extract the frames uniformly
extracted_frame = deeplabcut.extract_frames(path_config_file,  mode="manual") # Manual extraction of frames
plt.imshow(extracted_frame) # Plot the frame to make sure the cropping parameters work

## 3. Labelling the frames :
Now you can label the extracted frames with the bodypart labels you defined in the config file. The labeled images will be saved in the project directory in the sub-directory 'labeled-data'.

In [ ]:
%gui wx

deeplabcut.label_frames(path_config_file)
# At this point you can check the labels since this is the most cucial step for creating a training dataset.

deeplabcut.check_labels(path-config_file)
# You can now check the frames with your labels in the 'check-labels' subdirectory
# You can also adjust the labels by relaunching the labeling GUI and move the labels around and click save.

## 4. Creating a training dataset :
You can create a training dataset using the function provided below. This geenrates the training information that the network will use. You can adjust the training and test fraction in the config.yaml file by changing the 'TrainingFraction' parameter. You can also create multiple shuffles of the data to benchmark performance, the default is set to 1. There are also several networks that you can pick from, the default network used here is the resnet-50.

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## 5. Start training :

In [ ]:
deeplabcut.train_network(path_config_file)

## 6. Evaluate the network :
After the network has trained, you need to evaluate how your network has learnt. To do this, you can use the following function which will store the results as a .csv file and also plots the results.

In [ ]:
deeplabcut.evaluae_network(path_config_file, plotting=True)

## 7. Analyze your videos :
If you ae happy with the training of your network, you can now start analyzing your videos.

In [ ]:
# Provide the videos you want to analyze:

analyze_videopath = ['video/video1.mp4', 'video/video2.mp4']
deeplabcut.analyze_videos(path_config_file, analyze_videopath, videotype='.mp4')

## 7a. Extract outlier frames :

When the evaluation results are not satisfactory, and the labels are predicted poorly, then you can extract frames in which the labels are predicted incorrectly and refine them.

In [ ]:
deeplabcut.extact_outlier_frames(path_config_file, ['video/video1.mp4']) # You need to provide a specific video here

In [ ]:
%gui wx
deeplabcut.refine_labels(path_config_file) # This opens the GUI and allows you to adjust the markers in the newly extacted frames
deeplabcut.merge_datasets(pah_config_file) # This merges the new labels with the older labels

In [ ]:
# When the new labels are megred with the original dataset, there is a new iteration of training datasets created automatically and adjusted in the config.yaml file. You can now reate a training dataset and start training
deeplabcut.create_training_dataset(path_config_file)
deeplabcut.train_network(path_config_file)

## 8. Creating labeled videos :
Now you can visualize the fruits of your labour, the following function can be used to create .mp4 videos with the labels predicted by the network. The videos are saved in the same directory as the original videos.

In [ ]:
deeplabcut.create_labeled_video(path_config_file)

# Plot the trajectories of the analyzed vidoes
%matplotlib notebook
deeplabcut.plot_trajectories(path_config_file, analyz_videopath)